In [1]:
using Unitful

In [2]:
function test5(x,y,d,t)
    z = (x + y)*d/t
    return z
end

x = test5(1u"kg",1u"kg",2u"m",1u"s")

x * 5u"s"

20.0 kg m

In [3]:
module mod
    function y(x)
        y = 2*x
        return y
    end

    function z(x,y)
        z = x^2+y^3/2
        return z
    end

    function p(t,q)
        p = 2*q+t
        return p
    end

    function x(y,z)
        x = y^3/2-z^2
        return x
    end

    function q(x,y,z)
        q = x^2/1.5+z*4-y^2
        return q
    end
end

module test
    module mod2
        function displacement(v0, t, a)
            Δx = v0*t+1/2*a*t^2
            return Δx
        end

        function acceleration(v0, vf, dt)
            a = (vf-v0)/dt
            return a
        end

        function zerp(t,a)
            p = 2*a+t
            return p
        end

        function ac(z)
            ac = z
            return ac
        end
    end
end

Main.test

In [54]:
module constants
    using Unitful
    gEarth = 9.80665u"m/s^2"
    GEarth = (6.67408*10^(-11))u"N*m^2/kg^2"
    k = (8.9875517887*10^9)u"N*m^2/C^2"
    c = 299792458u"m/s"
    e = (1.60217662*10^(-19))u"C"
    h = (6.62607004*10^(-34))u"-34"
    eps0 = (8.85418782*10^(-12))u"m^(-3)*kg^(-1)*s^(4)*A^(2)"
    mu0 = (1.25663706*10^(-6))u"m*kg*s^-2*A^-2"
    mEarth = (5.972*10^24)u"kg"
    rEarth = 6378.1u"km"
    mSun = (1.989*10^30)u"kg"
    export
        gEarth,
        GEarth,
        k,
        c,
        e,
        h,
        eps0,
        mu0,
        mEarth,
        rEarth,
        mSun
end

module physics
    module kinematics
        module oneD
            # Displacement
            disp(x0,x) = x-x0
            dispConstAcc1(v0,a,t) = v0*t+1/2*a*t^2
            dispConstAcc2(v0,v,t) = ((v+v0)/2)*t
            # Velocity
            avgVel(x0,x,t0,t) = (x-x0)/(t-t0)
            velConstAcc1(v0,a,t) = v0+a*t
            velConstAcc2(x0,x,v0,a) = sqrt(v0^2+2*a*(x-x0))
            # Acceleration
            accConst(x0,x,v0,v) = (v^2-v0^2)/(2*(x-x0))
        end
        module projectile
            using ....constants
            range(v0,th0) = v0^2/gEarth*sin(2*th0)
            maxHeight(v0,th0) = v0^2/(2*gEarth)*sin(th0)^2
        end
    end
end

Main.physics

In [53]:
physics.kinematics.projectile.maxHeight(1u"m/s",2u"rad")

0.04215618026705379 m

In [44]:
# Displacement
disp(x0,x) = x-x0
dispConstAcc1(v0,a,t) = v0*t+1/2*a*t^2
dispConstAcc2(v0,v,t) = ((v+v0)/2)*t
# Velocity
# velAvg(x0,x,t0,t) = (x-x0)/(t-t0)
velConstAcc1(v0,a,t) = v0+a*t
velConstAcc2(x0,x,v0,a) = sqrt(v0^2+2*a*(x-x0))
# Acceleration
accConst(x0,x,v0,v) = (v^2-v0^2)/(2*(x-x0))

accConst (generic function with 1 method)

In [45]:
module reflection

function modName(mod::Module)
    if mod == Main
        mod_name = string(mod)
    else
        mod_name = string(mod)[6:end]
    end

    return mod_name
end

# check if a symbol is a nested module
issubmodule(m::Module, s::Symbol) = isa((@eval $(m).$(s)),Module) && !occursin(string(s),modName(m))
    
# get a list of all submodules of module m
function submodules(m::Module, mod_arr::Array{Module})
    modules = filter(x->issubmodule(m,x),names(m,all=true))
    push!(mod_arr,m)
    for mod in modules
        submodules((@eval $(m).$(mod)), mod_arr)
    end
end

function findArgs(func::Function)
    methods_str = methods(func)
    arg_ind_1 = findlast("(",string(methods_str))[1]+1
    arg_ind_2 = findlast(")",string(methods_str))[1]-1

    arg_s = string(methods_str)[arg_ind_1:arg_ind_2]
    args = []
    str = ""
    isArg = true

    for char in arg_s
        str = str
        if isArg == false && (char == ',' || char == ';')
            isArg = true
        elseif isArg == true
            if char == ',' || char == ';'
                push!(args,Symbol(strip(str)))
                str = ""
            elseif char == '=' || char == ':'
                isArg = false
                push!(args,Symbol(strip(str)))
                str = ""
            else
                str = string(str,char)
            end
        end
    end
    if str != ""
        push!(args,Symbol(strip(str)))
    end

    return args
end

function moduleDict(mod::Module, funcs::Dict)
    mod_name = modName(mod)
    
    for func in names(mod, all=true)
        if (func != :eval && func != :include && 
            func != Symbol(mod_name) && func != :Base &&
            func != :Core)
            if !occursin("#",string(func))
                funcType = string(typeof(@eval $(mod).$(func)))
                if occursin("typeof",funcType)
                    func_args = findArgs(@eval $(mod).$(func))
                    funcs[func] = func_args
                end
            end
        end
    end
    return funcs
end

function evalVariableInputs(mod::Module, func::Symbol, args::Array{Any})
    global output = 0
    args = tuple(args...)
    output = (@eval $(mod).$(func)($(args)...))
    
    return output
end

function varExplode(var)
    var_arr = []
    var_name = string(var)
    var_string = ""

    for c in var_name
        var_string = string(var_string,c)
        push!(var_arr,Symbol(var_string))
    end
    
    return var_arr
end

function varExplodeDict(var_dict::Dict)
    inp_dict = Dict()
    out_dict = Dict()
    for pair in var_dict
        out_dict[pair[1]] = varExplode(pair[1])
        for var in pair[2]
            inp_dict[var] = varExplode(var)
        end
    end
        
    return out_dict, inp_dict
end

function varMapArray(d::Dict,var::Symbol)
    map_array = []
    
    for pair in d
        for var_n in pair[2]
            if var_n == var
                push!(map_array, pair[1])
            end
        end
    end
    return map_array
end

function crossVarMap(d1::Dict, d2::Dict)
    d12 = Dict()
    d21 = Dict()
    
    for pair in d1
        map_array = []
        for var in pair[2]
            map_array = vcat(map_array, varMapArray(d2,var))
        end
        d12[pair[1]] = vcat(pair[1], map_array)
    end
    
    for pair in d2
        map_array = []
        for var in pair[2]
            map_array = vcat(map_array, varMapArray(d1,var))
        end
        d21[pair[1]] = vcat(pair[1], map_array)
    end
    return d12,d21
end

function argCheck(mod::Module, arg::Symbol, arg_arr::Array{Any}, inp_arr::Array{Any}, inp_map::Dict)
    valid_inputs = 0
    mods = [Main,mod]
    
    for arg_inp in inp_map[arg]
        for m in mods
            if isdefined(m,arg_inp) && !isa((@eval $(m).$(arg_inp)),Function)
                if @eval $(m).$(arg_inp) != nothing
                    push!(inp_arr, @eval $(m).$(arg_inp))
                    push!(arg_arr, arg_inp)
                    valid_inputs = 1
                    break
                end
            else
                valid_inputs = 0
            end
        end
        if valid_inputs == 1; break; end
    end
    
    return valid_inputs
end

function outputDict(funcs::Dict, outputs::Dict; mod::Module=Main, inp_map::Dict=Dict())
    valid_inputs = 0
    global inp = []
    global arg_arr = Array{Symbol}
    all_solved = false
    none_solved = false
    num_solved = 0
    global output = 0
    
    while all_solved == false && none_solved == false
        for func in funcs
            if !haskey(outputs,func[1]) && !occursin(string(func[1]),"outputDict")
                valid_inputs = 0
                inp = []
                arg_arr = []
                
                for arg in func[2]
                    valid_inputs = argCheck(reflection, arg, arg_arr, inp, inp_map)
                    
                    if arg == func[2][end] && valid_inputs == 1
                        try
                            output = evalVariableInputs(mod, func[1], inp)
                            println("Func: $(func[1])$(tuple(func[2]...))")
                            println("Inputs:")
                            for i = 1:length(arg_arr)
                                println("$(arg_arr[i]) = $(inp[i])")
                            end
#                             println("Input Args: $(tuple(arg_arr...))")
#                             println("Inputs: $(tuple(inp...))")
                            println("Output: $(output)\n")
                            num_solved+=1
                            outputs[func[1]] = output
                            out = string(func[1])
                            @eval $(Symbol(out))=output
                        catch
                            println("Func: $(func[1])$(tuple(func[2]...))")
                            println("Inputs:")
                            for i = 1:length(arg_arr)
                                println("$(arg_arr[i]) = $(inp[i])")
                            end
                            println("Output: Error\n")
                        end
                    elseif valid_inputs == 0
                        break
                    end
                end
            end
        end
        if num_solved == length(funcs)
            all_solved = true
        elseif num_solved == 0
            none_solved = true
        end
        num_solved = 0
    end
end

end

Main.reflection

In [46]:
function initArgCheck(funcs::Dict, init_args::Array{Any})
    for func in funcs
        for arg in func[2]
            if isdefined(Main,arg) && @eval Main.$(arg) != nothing
                push!(init_args,arg)
            end
        end
    end
end

function resetArgs(funcs::Dict, init_args::Array{Any})
    for func in funcs
        for arg in func[2]
            if !any(x->x==arg, init_args)
                @eval $(arg)=nothing
            end
        end
    end
end

resetArgs (generic function with 1 method)

In [47]:
physics.kinematics.oneD.disp(1.0u"m",2.0u"m")

1.0 m

In [48]:
x0 = 1.0u"m"
x = 1.0u"km"
v0 = 1.0u"m/s"
v = 5.0u"m/s"
th0 = 0.4u"rad"

mods = Module[]
reflection.submodules(physics, mods)
# mods = [Main]

for mod in mods
    in_mod = mod
    @show mod
    funcs = Dict()
    outputs = Dict()
    reflection.moduleDict(in_mod,funcs)
    if !isempty(funcs)
        println("::::MODULE $(mod)::::")
        out_d, in_d = reflection.varExplodeDict(funcs)
        @show in_d
        inp_map, out_map = reflection.crossVarMap(in_d,out_d)
        @show inp_map
        println()
        reflection.outputDict(funcs, outputs, mod=in_mod, inp_map=inp_map)
        @show outputs
        println()
    end
end

mod = Main.physics
mod = Main.physics.kinematics
mod = Main.physics.kinematics.oneD
::::MODULE Main.physics.kinematics.oneD::::
in_d = Dict{Any,Any}(:a=>Any[:a],:x0=>Any[:x, :x0],:v0=>Any[:v, :v0],:v=>Any[:v],:t0=>Any[:t, :t0],:t=>Any[:t],:x=>Any[:x])
inp_map = Dict{Any,Any}(:a=>Any[:a, :avgVel, :accConst],:x0=>Any[:x0],:v0=>Any[:v0, :velConstAcc1, :velConstAcc2],:v=>Any[:v, :velConstAcc1, :velConstAcc2],:t0=>Any[:t0],:t=>Any[:t],:x=>Any[:x])

Func: disp(:x0, :x)
Inputs:
x0 = 1.0 m
x = 1.0 km
Output: 999.0 m

Func: accConst(:x0, :x, :v0, :v)
Inputs:
x0 = 1.0 m
x = 1.0 km
v0 = 1.0 m s^-1
v = 5.0 m s^-1
Output: 0.012012012012012012 m s^-2

Func: velConstAcc2(:x0, :x, :v0, :a)
Inputs:
x0 = 1.0 m
x = 1.0 km
v0 = 1.0 m s^-1
accConst = 0.012012012012012012 m s^-2
Output: 5.0 m s^-1

outputs = Dict{Any,Any}(:disp=>999.0 m,:accConst=>0.012012 m s^-2,:velConstAcc2=>5.0 m s^-1)

mod = Main.physics.kinematics.projectile
::::MODULE Main.physics.kinematics.projectile::::
in_d = Dict{Any,Any}(:v0=>A

In [ ]:
https://github.com/ordovician/Airship.jl/tree/master/src

# Testing

In [ ]:
x = 5
v0 = 0
vf = 10
dt = 1
t = 5
a = 10
funcs = Dict()
outputs = Dict()
moduleDict(mod2,funcs)
out_d, in_d = varExplodeDict(funcs)
inp_map, out_map = crossVarMap(in_d,out_d)

num_solved = 0
for func in funcs
    println(func)
    if !haskey(outputs,func[1])
        valid_inputs = 0
        inp = []
        for arg in func[2]
            for arg_inp in inp_map[arg]
                if isdefined(Main,arg_inp) && @eval $(arg_inp) != nothing
                    push!(inp,@eval $(arg_inp))
                    println(inp)
                    valid_inputs = 1
                    break
                else
                    valid_inputs = 0
                end
            end

            if arg == func[2][end] && valid_inputs == 1
                output = evalVariableInputs(Main.mod2, func[1], inp)
                println("Func: $(func[1])")
                println("Args: $(func[2])")
                println("Inputs: $(inp)")
                println("Output: $(output)\n")
                num_solved+=1
                outputs[func[1]] = output
                out = string(func[1])
                @eval $(Symbol(out))=output
            elseif valid_inputs == 0
                break
            end
        end
    end
end

In [ ]:
function dr(a,b)
    d = a+b
    return d
end

function a1(c,p)
    a = 5*c
    return a
end

b = [1 2]
c = [1 2 3]
p = [1 2]

funcs = ["dr" "a1"]
args = ["1" "2"]

for func in funcs
    for arg in args
        println("Func: $(func) / Arg: $(arg)")
        for i in c
            a2 = a1(i)
            for j in b
                println(dr(a2,j))
            end
        end
    end
end

In [ ]:
funcs = Dict()
in_d = Dict()
in_map = Dict()
out_d = Dict()
out_map = Dict()
moduleDict(mod2,funcs)
out_d, in_d = varExplodeDict(funcs)
in_map, out_map = crossVarMap(in_d,out_d)
out_map

In [ ]:
## function storeJuliaFunctions
# Given a module or Julia file, parse out functions and their
# respective variables and store locally or in a file.



In [ ]:
## function findConnections
# Given a set of input variables and functions, find and store all
# functions with equivalent variables as inputs or outputs.


In [ ]:
## function autoSolve
# Given a stored collection of inputs and outputs, determine
# which functions have valid inputs recursively, solve
# for all outputs, and store the outputs.

In [ ]:
## script functionLearn
# Script to use deep learning for finding connections and
# features within engineering libraries, codes, and equations.

In [ ]:
## script functionStore
# Script to combine with functionLearn to store functions with
# inputs and outputs from useful libraries.

In [ ]:
## script systemBoltzmann
# Based on system telemtry data, build a model using a
# Deep Boltzmann Machine to capture connections.

In [ ]:
## script realityConverter
# Compare deep learning model to analytical model and
# find which equations are most likely driving physical
# explanations.

In [ ]:
function k(x,y,z)
    x^2+y^2+z^2
end

In [ ]:
g = "   function k(x,y,z)"

In [ ]:
ind_1 = findfirst("function ",g)[end]+1
ind_2 = findlast(isequal(')'),g)

if occursin("#",g)
    if findfirst("#",g)[1] < findfirst("function ",g)[1]
        print("Damn")
    end
end

In [ ]:
func_name = g[ind_1:ind_2]

In [ ]:
g2 = g1[1]
g3 = g1[3]
g4 = g1[5]
g5 = g1[7]
g6 = g1[6]

In [ ]:
k1 = getfield(Main,Symbol(g2))

In [ ]:
x = 5
y = 2

if isdefined(Main, Symbol(g3)) && isdefined(Main, Symbol(g4))&& isdefined(Main, Symbol(g5))
    k1(getfield(Main, Symbol(g3)),getfield(Main, Symbol(g4)),getfield(Main, Symbol(g5)))
end

In [ ]:
function x()
    return 1
end

x = 5